# MINI Interview Agent Proof-of-Concept

A minimal LangChain-powered agent for conducting MINI (Mini International Neuropsychiatric Interview) assessments.

## 1 Setup

Install required packages. Note: Colab will auto-restart after first install.

In [2]:
%pip install langchain openai tiktoken ipykernel langchain-openai

Note: you may need to restart the kernel to use updated packages.


## 2 Imports and Keys

In [2]:
import os
import json
import getpass
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

def get_key():
    """Get OpenAI API key from environment or user input."""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        api_key = getpass.getpass('Enter your OpenAI API key: ')
        os.environ['OPENAI_API_KEY'] = api_key
    return api_key

# Set up API key
get_key()
print("API key configured")

API key configured


## 3 Load MINI Questions (Module E)

In [ ]:
# Load module data
import json, pprint

PATH = "mini_modules/working/module_e.json" 
with open(PATH, "r", encoding="utf-8") as f:
    module_data = json.load(f)

mini_script = module_data["questions"]     
module_info = module_data["module"]          

# Prepare questions dictionary for easy access
questions = {q["id"]: q for q in mini_script}

print(f"Loaded {len(mini_script)} questions for Module {module_info['id']}: {module_info['name']}\n")

for i, q in enumerate(mini_script[:2], start=1):
    print(f"{i}. {q['id']}: {q['prompt']}\n")

# Branching logic retreival function
def show_branches(qid: str):
    if qid not in questions:
        raise KeyError(f"{qid!r} not found")
    next = questions[qid].get("next", {})
    print(f"Branching for {qid}:")
    pprint.pp(next)

# Example: inspect the first question’s branches
show_branches("E1")

Loaded 6 questions for Module E: Agoraphobia

1. E1: Do you feel anxious or uneasy in places or situations where help might not be available or escape difficult if you had a panic-like or embarrassing symptom, such as being in a crowd or queue, in an open space or crossing a bridge, in an enclosed space, when alone away from home, when alone at home, or traveling in a bus, train, car, or using public transportation?

2. E2: Do these situations almost always bring on fear or anxiety?

Branching for E1:
{'yes': 'E2', 'no': 'END_MODULE'}


## 4 Prompt Template

In [ ]:
QUESTION_PROMPT = PromptTemplate(
    input_variables=["history", "question"],
    template="""You are an AI agent with the role of a skilled clinician trained in professional physchological diagnosis and care conducting an interview based on the MINI questionnaire. Be concise, empathetic, and professional.

Previous conversation:
{history}

Next question to ask:
{question}

Present this question in a natural, conversational way. Keep your response brief and focused, ask the question prompting a yes or no response from the patient."""
)

## 5 Chain Constructor

In [ ]:
# Build LangChain Agent
def build_interview_agent(llm):
    return LLMChain(
        llm=llm,
        prompt=QUESTION_PROMPT,
        verbose=False
    )

## 6 Interview Loop

In [ ]:
# Execute MINI interview simulation

def run_interview():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    agent = build_interview_agent(llm)
    
    conversation_log = []
    history = ""
    
    print(f"Starting MINI Module {module_info['id']}: {module_info['name']}\n")
    print("=" * 60)
    
    for i, question_data in enumerate(mini_script):
        question_id = question_data["id"]
        question_text = question_data["prompt"]
        
        print(f"\n[Question {question_id}]")
        
        # Generate clinician response
        response = agent.run(
            history=history,
            question=question_text
        )
        
        print(f"Clinician: {response}")
        
        # Patient response
        patient_answer = input(f"Type your answer:")

        history += f"Q{question_id}: {response}\nA: {patient_answer}\n\n"

        # Evaluate next step based on patient answer
        branching_logic = show_branches(question_id)
        next_question = agent.run(
            history=history,
            question=f"Based on the patient's response: {patient_answer} to {question_id}: {question_text}, use {branching_logic} to evaluate which question from should you ask next"
        )

        # Update conversation log and history
        conversation_log.append({
            "question_id": question_id,
            "clinician": response,
            "patient": patient_answer
        })
    
    print("\n" + "=" * 60)
    print(f"Interview completed. Total exchanges: {len(conversation_log)}")
    
    return conversation_log

## 7 Demo Run

In [8]:
# Execute the interview
results = run_interview()

/var/folders/5r/2h_9fsqs71s8v9wdpjjb8fb40000gn/T/ipykernel_58261/2420036983.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(
/var/folders/5r/2h_9fsqs71s8v9wdpjjb8fb40000gn/T/ipykernel_58261/910833007.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(


Starting MINI Module E: Agoraphobia


[Question E1]
Clinician: I appreciate you sharing your thoughts with me. I’d like to ask about your feelings in certain situations. Do you ever feel anxious or uneasy in places where help might not be available, or if you experienced panic-like symptoms, such as being in a crowd, an open space, or while traveling? Just a simple yes or no is fine.
Branching for E1:
{'yes': 'E2', 'no': 'END_MODULE'}

[Question E2]
Clinician: Thank you for your openness. I’d like to know, do these situations almost always bring on feelings of fear or anxiety for you? Just a simple yes or no is fine.
Branching for E2:
{'yes': 'E3', 'no': 'END_MODULE'}

[Question E3]
Clinician: Thank you for sharing that with me. I’d like to understand a bit more about how you cope. Do you find that you fear these situations so much that you either avoid them, endure them with discomfort, or feel you need someone with you to face them? Just a simple yes or no is fine.
Branching for E3:


## 8 Next Steps

Potential enhancements for this proof-of-concept:

- Add diagnosis
- Add threat detection (e.g. patient has suicidal thoughts)
- Add persistent storage for interview results and scoring
- Add multi module function
